In [2]:
import pandas as pd
import numpy as np
import os
import utils

In [3]:
path = "./tctodd/"
#path = "../../Desktop/MML Project/tctodd/"
dirs = os.listdir(path=path)
weeks = sorted([i for i in dirs if i != ".DS_Store"])
filenames = sorted(os.listdir(path+weeks[1]))

data = []
labels = dict()
label_cnt = 0

for w in weeks:
    temp_path = path+w+"/"
    filenames = sorted(os.listdir(temp_path))
    for fn in filenames:
        label = fn.split('.')[0][:-2]
        
        if label not in labels:
            labels[label] = label_cnt
            label_cnt += 1
            
        data.append({'label':labels[label], 'time_series':pd.read_csv(temp_path+fn, header=None, sep='\t',).values})
        

In [4]:
df = pd.DataFrame(data, columns=['label', 'time_series'])
seed = 0

In [5]:
X = df['time_series']
y = df['label']

n_coords = 60
X_interp = utils.interpolate_data(X, n_coords)
X_interp_concatenated = utils.concatenate_examples(X_interp)

print(f"Shape of new interpolated and stacked X is {X_interp.shape}")

Shape of new interpolated and stacked X is (2565,)


# TimeSeriesForest

In [56]:
from sktime.classification.interval_based import TimeSeriesForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_interp_concatenated, y,
                                                    stratify=y, 
                                                    test_size=0.1, random_state=seed)

ts_forest = TimeSeriesForestClassifier(random_state=seed)

ts_forest.fit(X_train, y_train)
preds = ts_forest.predict(X_test)
score = accuracy_score(y_test, preds)
print(f"Score of the plain TimeSeriesForestClassifier without cross-validation is {score}")

Score of the plain TimeSeriesForestClassifier without cross-validation is 0.6264591439688716


-------
# 1-Nearest Neighbor + DTW

In [1]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
import time
import os

X

NameError: name 'X' is not defined

# NN-DTW with interpolated data

In [55]:
from sktime.classification.distance_based import KNeighborsTimeSeriesClassifier
import time
import os

# to match KNeighborsTimeSeriesClassifier requirements, we need predictors x time instants
X_interp_new = X_interp.apply(lambda x: x.T)
print("The new shape is", X_interp_new[0].shape)

X_train, X_test, y_train, y_test = train_test_split(X_interp_new, y,
                                                    stratify=y, 
                                                    test_size=0.1, random_state=seed)
X_train_3d = np.stack([i for i in X_train])
X_test_3d = np.stack([i for i in X_test])

nn_dtw = KNeighborsTimeSeriesClassifier()


t0_fit = time.time()
nn_dtw.fit(X_train_3d, y_train)
t1_fit = time.time() - t0_fit

os.system(f'say "Ci ha impiegato {t1_fit} secondi per fittare 1-NeighborTimeSeriesClassifier"')

print("Starting prediction")
t0_pred = time.time()
preds = nn_dtw.predict(X_test_3d)
t1_pred = time.time() - t0_pred
score = accuracy_score(y_test, preds)
os.system(f'say "Ci ha impiegato {t1_pred} secondi per predire"')

print(f"Score of the plain 1-NeighborTimeSeriesClassifier without cross-validation is {score}")

os.system(f'say "Accuracy calcolata pari a {score}. Ci sono voluti {t1_fit + t1_pred} secondi per fittare e predire."')

The new shape is (22, 60)
Starting prediction
Score of the plain 1-NeighborTimeSeriesClassifier without cross-validation is 0.8132295719844358


0

--------
# Time Series SVC (to check)

In [29]:
from tslearn.svm import TimeSeriesSVC

n_coords = 60
X_interp = utils.interpolate_data(X, n_coords)
X_interp.shape

(2565,)

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X_interp, y,
                                                    stratify=y, 
                                                    test_size=0.1, random_state=seed)
X_train_3d = np.stack([i for i in X_train])
X_test_3d = np.stack([i for i in X_test])


In [43]:
X_train_3d.shape, X_test_3d.shape

((2308, 60, 22), (257, 60, 22))

In [41]:
ts_svc = TimeSeriesSVC(random_state=seed)
ts_svc.fit(X_train_3d, y)

preds = ts_svc.predict(X_test_3d)

score = accuracy_score(y_test, preds)

print(score)

ValueError: Found input variables with inconsistent numbers of samples: [2308, 2565]

In [44]:
from tslearn.generators import random_walk_blobs
X, y = random_walk_blobs(n_ts_per_blob=10, sz=64, d=2, n_blobs=2)

In [45]:
X.shape

(20, 64, 2)

In [46]:
X

array([[[ -1.24371017,   2.8175088 ],
        [ -0.3578046 ,  -2.20753332],
        [  1.41337159,  -0.73974462],
        ...,
        [ -3.36882881, -21.20823503],
        [ -3.29372155, -20.21661323],
        [ -4.81592338, -20.36023248]],

       [[ -0.2575385 ,  -0.62689972],
        [  0.77218854,   0.73222972],
        [ -0.24277063,   1.78714711],
        ...,
        [ -1.79575173, -19.73295393],
        [ -4.36792466, -21.28876755],
        [ -3.45534101, -21.96970223]],

       [[  0.98962533,   0.09721761],
        [ -0.95298098,  -1.07086234],
        [  1.54133056,   0.47432605],
        ...,
        [ -2.67400058, -18.7400684 ],
        [ -2.36978663, -21.46818139],
        [ -4.88710341, -21.71974484]],

       ...,

       [[  2.32789245,  -2.00302668],
        [  1.10170244,  -1.58934998],
        [  2.49968093,  -0.21379147],
        ...,
        [  0.41458057,   2.58617679],
        [  1.19690119,   3.31199788],
        [ -0.56349961,   3.29777117]],

       [[  1.51